In [1]:
from functools import partial
import json
import numpy as np
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import random_split
import torchvision
import torchvision.transforms as transforms
from ray import tune
from ray.tune import JupyterNotebookReporter
from ray.tune.schedulers import ASHAScheduler

from my_models import (AlexNet, VGG16, ResNet)

In [2]:
in_ch = 3
out_ch = 10

models_list = [AlexNet, VGG16, ResNet]

In [3]:
# создание папок для логов разых моделей:
checkpoint_dir="./data/checkpoints/"

for name in models_list:    
    if not os.path.isdir(checkpoint_dir+str(name)):
        os.makedirs(checkpoint_dir+str(name))


In [4]:
# исходники: https://pytorch.org/tutorials/beginner/hyperparameter_tuning_tutorial.html

In [5]:
def load_data(data_dir="./data/CIFAR"):
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    trainset = torchvision.datasets.CIFAR10(
        root=data_dir, train=True, download=True, transform=transform)

    testset = torchvision.datasets.CIFAR10(
        root=data_dir, train=False, download=True, transform=transform)

    return trainset, testset

In [6]:
def train_cifar(config, model_name, epoch_num=2,
                checkpoint_dir=checkpoint_dir, data_dir=None): 
    
    net = model_name(in_ch, out_ch)
    
    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda:0"
        if torch.cuda.device_count() > 1:
            net = nn.DataParallel(net)

    net.to(device)
    
    criterion = config["losses"]() # для итерирования разных losses
    optimizer = config["optimizers_names"](net.parameters(), lr=config["lr"])
    """
    if checkpoint_dir: 
        model_state, optimizer_state = torch.load(
            os.path.join(checkpoint_dir+model_name, "checkpoint"))
        net.load_state_dict(model_state)
        optimizer.load_state_dict(optimizer_state)
    """
    trainset, testset = load_data(data_dir)

    test_abs = int(len(trainset) * 0.8)
    train_subset, val_subset = random_split(
        trainset, [test_abs, len(trainset) - test_abs])

    trainloader = torch.utils.data.DataLoader(
        train_subset,
        batch_size=int(config["batch_size"]),
        shuffle=True)
    valloader = torch.utils.data.DataLoader(
        val_subset,
        batch_size=int(config["batch_size"]),
        shuffle=True)

    for epoch in range(epoch_num):  # loop over the dataset multiple times

        for i, data in enumerate(trainloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        # Validation loss
        val_loss = 0.0
        val_steps = 0
        total = 0
        correct = 0
        for i, data in enumerate(valloader, 0):
            with torch.no_grad():
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = net(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

                loss = criterion(outputs, labels)
                val_loss += loss.cpu().numpy()
                val_steps += 1

        with tune.checkpoint_dir(epoch) as checkpoint_dir:
            path = os.path.join(checkpoint_dir, "checkpoint")
            torch.save((net.state_dict(), optimizer.state_dict()), path)

        tune.report(loss=(val_loss / val_steps), accuracy=correct / total)
    print("Finished Training")

In [7]:
def main(model_name, num_samples=10, max_num_epochs=10, gpus_per_trial=2):
    data_dir = os.path.abspath("./data")
    load_data(data_dir)
    
    # заменила конфиг
    config = {        
        "lr":tune.grid_search([1e-2, 1e-1]),   
        "batch_size": tune.grid_search([100, 1000]),
        "optimizers_names":  tune.grid_search([optim.Adam, optim.SGD]), # ключи словаря
        "losses": nn.CrossEntropyLoss 
    }
    scheduler = ASHAScheduler(
        metric="loss",
        mode="min",
        max_t=max_num_epochs,
        grace_period=1,
        reduction_factor=2)
    reporter = JupyterNotebookReporter(
        # parameter_columns=["l1", "l2", "lr", "batch_size"],
        overwrite = True,
        print_intermediate_tables = False,
        metric_columns=["loss", "accuracy", "precision"]) # немного побаловалась, не нашла что еще добавить на посмотреть
    result = tune.run(
        partial(train_cifar, data_dir=data_dir, model_name=model_name,
                checkpoint_dir=checkpoint_dir),
        config=config,
        num_samples=num_samples,
        scheduler=scheduler,
        progress_reporter=reporter,
    local_dir= checkpoint_dir+str(model_name))
    
    best_trial = result.get_best_trial("loss", "min", "last")
    print("Best trial config: {}".format(best_trial.config))
    print("Best trial final validation loss: {}".format(
        best_trial.last_result["loss"]))
    print("Best trial final validation accuracy: {}".format(
        best_trial.last_result["accuracy"]))    

    return best_trial   

In [8]:
for model_name in models_list:
    best_trial = main(model_name, num_samples=1, max_num_epochs=2, gpus_per_trial=1)
    best_trial_conf = best_trial.config
    best_trial_conf['path']=best_trial.checkpoint.value
    # файлик, который говорит, где лежит чекпоинт нужной модели и какие параметры:

    with open("./data/best_trials_info/best_trial_dir_{}.txt".format(str(model_name)), "w") as file:        
        json.dump(best_trial_conf, file,  default=lambda o: str(o))

Trial name,status,loc,batch_size,lr,optimizers_names,loss,accuracy
DEFAULT_483fd_00000,TERMINATED,,100,0.01,<class 'torch.optim.adam.Adam'>,1.22226,0.555
DEFAULT_483fd_00001,TERMINATED,,1000,0.01,<class 'torch.optim.adam.Adam'>,1.55684,0.4253
DEFAULT_483fd_00002,TERMINATED,,100,0.1,<class 'torch.optim.adam.Adam'>,2.30584,0.1062
DEFAULT_483fd_00003,TERMINATED,,1000,0.1,<class 'torch.optim.adam.Adam'>,2.43262,0.1088
DEFAULT_483fd_00004,TERMINATED,,100,0.01,<class 'torch.optim.sgd.SGD'>,2.2455,0.1984
DEFAULT_483fd_00005,TERMINATED,,1000,0.01,<class 'torch.optim.sgd.SGD'>,2.30029,0.104
DEFAULT_483fd_00006,TERMINATED,,100,0.1,<class 'torch.optim.sgd.SGD'>,1.7792,0.3104
DEFAULT_483fd_00007,TERMINATED,,1000,0.1,<class 'torch.optim.sgd.SGD'>,2.28716,0.1461


2020-12-30 10:27:17,866	INFO tune.py:448 -- Total run time: 133.07 seconds (133.04 seconds for the tuning loop).


Best trial config: {'lr': 0.01, 'batch_size': 100, 'optimizers_names': <class 'torch.optim.adam.Adam'>, 'losses': <class 'torch.nn.modules.loss.CrossEntropyLoss'>}
Best trial final validation loss: 1.2222557097673417
Best trial final validation accuracy: 0.555
